
<p><img src="https://oceanprotocol.com/static/media/banner-ocean-03@2x.b7272597.png" alt="drawing" width="800" align="center"/>


<h1><center>Ocean Protocol - Manta Ray project</center></h1>
<h3><center>Decentralized Data Science and Engineering, powered by Ocean Protocol</center></h3>
<p>Version 0.4 - alpha preview</p>
<p><a href="https://github.com/oceanprotocol/mantaray">mantaray on Github</a></p>
<p>

Getting Underway - Downloading Datasets (Assets)
In this notebook, TODO: description

### Section 0: Import modules, and setup logging

In [1]:
# Standard imports
import logging
from pprint import pprint
import os
from pathlib import Path
import random
# Import mantaray and the Ocean API (squid)
import squid_py
from squid_py.ocean.ocean import Ocean
from squid_py.config import Config
import mantaray_utilities as manta_utils
from squid_py.keeper.web3_provider import Web3Provider
# Setup logging
manta_utils.logging.logger.setLevel('INFO')
from mantaray_utilities.user import password_map
# manta_utils.logging.logger.setLevel('DEBUG')
# os.environ['USE_K8S_CLUSTER'] = 'True' # Enable this for testing local -> AWS setup

10 - logging         - <module>        - Logging started


Using default logging settings.


In [2]:
# Get the configuration file path for this environment
CONFIG_INI_PATH = manta_utils.config.get_config_file_path()
logging.critical("Deployment type: {}".format(manta_utils.config.get_deployment_type()))
logging.critical("Configuration file selected: {}".format(CONFIG_INI_PATH))
logging.critical("Squid API version: {}".format(squid_py.__version__))

50 - <ipython-input-2-84136f362abd> - <module>        - Deployment type: USE_K8S_CLUSTER
50 - <ipython-input-2-84136f362abd> - <module>        - Configuration file selected: /home/batman/ocn/mantaray_jupyter/config_k8s_deployed.ini
50 - <ipython-input-2-84136f362abd> - <module>        - Squid API version: 0.4.3


### Section 1: Instantiate a simulated User
A 'User' in an abstract class representing a user of Ocean Protocol


In [3]:
configuration = Config(CONFIG_INI_PATH)
ocn = Ocean(configuration)

20 - ocean_secret_store - __init__        - 	SecretStore: url https://secret-store.dev-ocean.com, parity-client https://nile.dev-ocean.com, account 0x413c9ba0a05b8a600899b41b0c62dd661e689354
20 - diagnostics     - verify_contracts - Keeper contract artifacts (JSON abi files) at: /home/batman/ocn/mantaray_jupyter/artifacts_nile
20 - diagnostics     - verify_contracts - Using keeper contracts from network nile, network id is 8995
20 - diagnostics     - verify_contracts - Looking for keeper contracts ending with ".nile.json", e.g. "ServiceExecutionAgreement.nile.json".
20 - diagnostics     - verify_contracts - Finished loading keeper contracts:
	Dispenser: 0xb8B0ec3AC0bf28ebB47b3cce4b1B7607DD7FA2DB
	OceanToken: 0x88CAA68F41DD7cFdD431BcA036E11bd20ef58882
	DIDRegistry: 0x9d306Ca587ff4b311C7963e62F48f3d6B59Ec1a1
	ServiceExecutionAgreement: 0xFfCb6bea15BbF19Dd3bCDC82f1864A92F359284A
	PaymentConditions: 0xc00b256Ff109EDaA5A375799CFB7386221863329
	AccessConditions: 0x1be580a31D79a7FAcF1F5c70d8F

In [4]:
# Get a consumer account
path_passwords = manta_utils.config.get_project_path() / 'passwords.csv'
passwords = manta_utils.user.load_passwords(path_passwords)

consumer_acct = random.choice([acct for acct in ocn.accounts.list() if password_map(acct.address, passwords)])
consumer_acct.password = password_map(consumer_acct.address, passwords)
assert consumer_acct.password
print("Consumer account address: ", consumer_acct.address)

20 - user            - load_passwords  - 31 account-password pairs loaded


Consumer account address:  0x45234B8c216231c2c31785aE4F5DceB858A8Ec71


### Section 2: Find an asset

In [5]:
# Use the Query function to get all existing assets
basic_query = {"service":{"$elemMatch":{"metadata": {"$exists" : True }}}}
all_ddos = ocn.assets.query(basic_query)
assert len(all_ddos), "There are no assets registered, go to s03_publish_and_register!"
print("There are {} assets registered in the metadata store.".format(len(all_ddos)))

assert len(all_ddos), "There are no assets registered, go to s03_publish_and_register!"

# Get a DID for testing
selected_did = all_ddos[-1].did
print("Selected DID:",selected_did)

20 - ocean_assets    - query           - Searching asset query: {'service': {'$elemMatch': {'metadata': {'$exists': True}}}}


There are 67 assets registered in the metadata store.
Selected DID: did:op:8ed86f54d32c4180bb220467a0320655e17818549f304969992ba7e1a699140b


In [6]:
#TODO: The Asset class does not offer much beyond DDO class
#TODO: Term 'asset' is confusing here
this_asset = ocn.assets.resolve(selected_did)
#pprint(this_asset)
print(this_asset.metadata['base']['name'])
print("Price:", this_asset.metadata['base']['price'])

Ocean protocol white paper
Price: 9


Your account will need some Ocean Token to make real transactions

In [7]:
if ocn.accounts.balance(consumer_acct).ocn < this_asset.metadata['base']['price']:
    ocn.accounts.request_tokens(consumer_acct, 100)

Purchase the Asset!

In [8]:
#TODO: The service_definition_id will change to service_type
service_agreement_id = ocn.assets.order(this_asset.did, 'Access', consumer_acct)
print('New service agreement id:', service_agreement_id)

New service agreement id: 0x47558e6e12474d4e8200f456e904454ce828d3e0fc1e41ceacea9805f049c607


The asset download is automatically initiated, this will take time to complete!

In [9]:
asset_path = Path.cwd() / ocn._config.downloads_path / f'datafile.{this_asset.asset_id}.0'
print("Check for your downloaded asset in", asset_path)
print("This might not appear immediately - the transaction needs be mined and the download needs to complete!")

Check for your downloaded asset in /home/batman/ocn/mantaray_jupyter/downloads_nile/datafile.8ed86f54d32c4180bb220467a0320655e17818549f304969992ba7e1a699140b.0
This might not appear immediately - the transaction needs be mined and the download needs to complete!


20 - utils           - process_tx_receipt - Success: got PaymentLocked event after fulfilling condition.
20 - brizo           - consume_service - invoke consume endpoint with this url: https://nginx-brizo.dev-ocean.com//api/v1/brizo/services/consume?url=https://testocnfiles.blob.core.windows.net/testfiles/testzkp.pdf&serviceAgreementId=0x47558e6e12474d4e8200f456e904454ce828d3e0fc1e41ceacea9805f049c607&consumerAddress=0x45234B8c216231c2c31785aE4F5DceB858A8Ec71
20 - serviceAgreement - fulfill_agreement - About to do fulfillAgreement: account 0x45234B8c216231c2c31785aE4F5DceB858A8Ec71, saId 0x47558e6e12474d4e8200f456e904454ce828d3e0fc1e41ceacea9805f049c607, ServiceAgreement address 0xFfCb6bea15BbF19Dd3bCDC82f1864A92F359284A
20 - brizo           - consume_service - Total size of testzkp.pdf: 0 bytes.
495544KB [00:01, 251218.24KB/s]20 - brizo           - consume_service - Saved downloaded file in /home/batman/ocn/mantaray_jupyter/downloads_nile/datafile.8ed86f54d32c4180bb220467a0320655e1781